In [17]:
from Bio import SeqIO

# Open the fasta file and read sequences
salmo_chrome = r"C:\Users\User\Downloads\Salmonella_enterica.txt"  # Replace with your file path

In [18]:
def readGenome(salmo_chrome):
    genome = ''
    with open(salmo_chrome, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [19]:
genome = readGenome(salmo_chrome)

In [29]:
def pattern_count(text, pattern):
    count = 0
    text_length = len(text)
    pattern_length = len(pattern)

    for i in range(text_length - pattern_length + 1):
        if text[i:i + pattern_length] == pattern:
            count += 1
    return count

In [11]:
text = 'CGCGATACGTTACATACATGATAGACCGCGCGCGATCATATCGCGATTATC'
pattern = 'CGCG'
print(pattern_count(text, pattern)) 

5


In [5]:
text = 'ATGCGCGTA'
pattern = 'GCG'
print(pattern_count(text, pattern)) 

2


In [6]:
text = 'AAAGAGTGTCTGA'
pattern = 'AAA'
print(pattern_count(text, pattern)) 

1


In [7]:
text = 'AGCGTGCCGAAATTT'
pattern = 'TTT'
print(pattern_count(text, pattern)) 

1


In [8]:
text = 'GGACTTACTGACGTACG'
pattern = 'ACT'
print(pattern_count(text, pattern)) 

2


In [10]:
text = 'GGTTCGTTAACCTCGGTGTGCTAAAACTGTTAGCATGCCCCAAACTGTAAACTGTCCGAAAAACTGTAAACTGTGCAAACTGTGAAAAACTGTAAACTGTGCGCGCCCAGATAAAACTGTAAACTGTAAAACTGTGTAAACTGTATGAAACTGTAAACTGTTCCTAAACTGTAGGTGAAACTGTAAACTGTAAACTGTCTATAAAACTGTAAACTGTTGGAAACTGTAAAAACTGTTTAAAACTGTAAACTGTTGGAAACTGTAAACTGTGTCCGTCCGAAAAACTGTTAAACTGTAAAACTGTGGAAAACTGTCGAAAACTGTAAACTGTCAAACTGTATCGATAAACTGTAAAACTGTTTCCAAACTGTTAAACTGTCAAACTGTCGAAAACTGTCCAAACTGTGGCAAACTGTTCAAACTGTATTAAACTGTTAAACTGTAAACTGTAAACTGTAAACTGTGAAACTGTAAACTGTCCGCACTAAACTGTACAATCAAACTGTAAACTGTCCTCACTAGAAACTGTAAACTGTAAACTGTGAAACTGTGGCAAACTGTAAACTGTAAACTGTAAACTGTAAACTGTTAAACTGTGCACCAAACTGTACGACAGGCAGTAGAAACTGTAAAACTGTATGACGCAAACTGTCAAACTGTTCCTAAACTGTGAAACTGTAGCGCGAAACTGTTCAAACTGTTAAACTGTCAAACTGTCTTAGAAAACTGTGAAACTGTAAAACTGTATTATACTTCCTGAAACTGTCACGTAAACTGTGAAACTGTGATAAACTGTTCAGGGTAATATAAACTGTAAACTGTCCAAACTGTTAGCGAAACTGTAAACTGTACAAACTGTAAACTGTATTCAACAAAACTGTAAACTGTTGATAGTGAAACTGTAAACTGTACAAACTGTGGAAACTGTTCCTCAAACTGTAACAAACTGT'
pattern = 'AAACTGTAA'
print(pattern_count(text, pattern)) 

34


In [32]:
def frequent_words(text, k):
    frequent_patterns = set()  # Use a set to automatically remove duplicates
    counts = []
    text_length = len(text)

    for i in range(text_length - k + 1):
        pattern = text[i:i + k]
        counts.append(pattern_count(text, pattern))  # Use the pattern_count function

    max_count = max(counts)

    for i in range(text_length - k + 1):
        if counts[i] == max_count:
            frequent_patterns.add(text[i:i + k])

    return frequent_patterns

In [33]:
def frequency_table(text, k):
    freq_map = {}
    n = len(text)
    for i in range(n - k + 1):
        pattern = text[i:i + k]
        if pattern not in freq_map:
            freq_map[pattern] = 1
        else:
            freq_map[pattern] += 1
    return freq_map

In [34]:
def clump_finder(text, k, L, t):
    
    frequent_k_mers = set()
    n = len(text)
    if n < L:
        return frequent_k_mers

    # 1. Compute the frequency table for the first window
    first_window = text[:L]
    freq_map = frequency_table(first_window, k)

    # 2. Check for clumps in the first window
    for kmer, count in freq_map.items():
        if count >= t:
            frequent_k_mers.add(kmer)

    # 3. Slide the window and update the frequency table
    for i in range(1, n - L + 1):
        # Remove the k-mer that is leaving the window
        old_kmer = text[i - 1:i - 1 + k]
        freq_map[old_kmer] -= 1
        if freq_map[old_kmer] == 0:
            del freq_map[old_kmer]  # Remove k-mer if its count reaches 0

        # Add the k-mer that is entering the window
        new_kmer = text[i + L - k:i + L]
        if new_kmer in freq_map:
            freq_map[new_kmer] += 1
        else:
            freq_map[new_kmer] = 1

        # Check for clumps in the current window
        if freq_map[new_kmer] >= t:
            frequent_k_mers.add(new_kmer)

    return frequent_k_mers

In [45]:
text = genome[3751756:3758256]
k = 9
L = 500
t = 3
clumps = clump_finder(text, k, L, t)
print(f"k-mers forming clumps: {clumps}")

k-mers forming clumps: {'AACGTTTCG', 'CGCCCAGCG'}


In [23]:
def hamming_distance(str1, str2):
    
    if len(str1) != len(str2):
        return -1  # Hamming distance is only defined for equal-length strings
    distance = 0
    for i in range(len(str1)):
        if str1[i] != str2[i]:
            distance += 1
    return distance

def approximate_pattern_count(text, pattern, d):
    """
    Counts the number of approximate occurrences of a pattern in a text,
    allowing up to d mismatches.

    Args:
        text: The text string to search within.
        pattern: The pattern string to search for.
        d: The maximum number of mismatches allowed.

    Returns:
        The number of times the pattern appears in the text with at most d mismatches.
    """
    count = 0
    for i in range(len(text) - len(pattern) + 1):
        pattern_prime = text[i:i + len(pattern)]  # Extract substring of text
        if hamming_distance(pattern, pattern_prime) <= d:
            count += 1
    return count

In [14]:
def hamming_distance(str1, str2):
    """
    Computes the Hamming distance between two strings.

    Args:
        str1: The first string.
        str2: The second string.

    Returns:
        The Hamming distance between the two strings.
        Returns -1 if the strings have different lengths.
    """
    if len(str1) != len(str2):
        return -1  # Hamming distance is only defined for equal-length strings
    distance = 0
    for i in range(len(str1)):
        if str1[i] != str2[i]:
            distance += 1
    return distance

def approximate_pattern_count(text, pattern, d):
    """
    Counts the number of approximate occurrences of a pattern in a text,
    allowing up to d mismatches.

    Args:
        text: The text string to search within.
        pattern: The pattern string to search for.
        d: The maximum number of mismatches allowed.

    Returns:
        The number of times the pattern appears in the text with at most d mismatches.
    """
    count = 0
    for i in range(len(text) - len(pattern) + 1):
        pattern_prime = text[i:i + len(pattern)]  # Extract substring of text
        if hamming_distance(pattern, pattern_prime) <= d:
            count += 1
    return count

In [48]:
text = 'CGTGACAGTGTATGGGCATCTTT'
pattern = 'TGT'
d = 1
print(approximate_pattern_count(text, pattern, d))

8
